# Overfitting as a Concept

Overfitting means the model fits the parameters too closely with regard to the particular observations in the training dataset but does not generalize well to new data; we say that the model has a high variance. The reason for the overfitting is that our model is too complex for the given training data. Common solutions to reduce the generalization error are as follows:
 - Collect more training data
 - Introduce a penalty for complexity via regularization 
 - Choose a simpler model with fewer parameters
 - Reduce the dimensionality of the data
 
### L1 and L2 Regularization Penalties

We penalize large individual weights by adding the L1 or L2 norm of our weight vector to our loss function.
$$
L2: \Vert \mathbf{w} \Vert_2^2 = \sum_{j=1}^m w_j^2 \\
L1: \Vert \mathbf{w} \Vert_1 = \sum_{j=1}^m |w_j|
$$
In contrast to L2 regularization, L1 regularization usually yields sparse feature vectors, and most fea- ture weights will be zero. Sparsity can be useful in practice if we have a high-dimensional dataset with many features that are irrelevant, especially in cases where we have more irrelevant dimensions than training examples. In this sense, L1 regularization can be understood as a technique for feature selection.

Since the contours of an L1 regularized system are sharp, it is more likely that the optimum—that is, the intersection between the ellipses of the loss function and the boundary of the L1 diamond—is located on the axes, which encourages sparsity.

# Assessing Model Performance

In general, it's common to use the *holdout method* to assess the performance of a model, which involves performing a train-test split and training the model on the training set, and then estimating its generalization ability by its performance on the test set. However, we often want to perform *model selection*, wherein we find the optimal model for the dataset by tuning its hyperparameter values.

To accomplish this, we need to have a pseudo-test set taken from the training set on which to evaluate the different hyperparameter combinations; we call this set of data the *validation set*. Unfortunately, the success of this approach is very dependent on how we choose the validation set. So, we often decide to perform the much more robust method of *$k$-fold cross-validation*. The method is as follows:
1. Perform a train-test split on the dataset.
2. Randomly partition the training set into $k$ folds.
3. Use $k-1$ folds to train the dataset on all sets of hyperparameters.
4. Repeat step 3 $k$ times, using each fold as the validation set exactly once.
5. Find the optimal set of hyperparameters by calculating the average performance of each set of hyperparameters over all folds $F_i$:
$$
E = \frac{1}{k}\sum_{i=1}^k E_i.
$$

In general, we choose $k=10$ folds as this has empirically been shown to provide the best trade-off between bias and variance. However, when the dataset is small, it may make sense to choose a larger value of $k$ so that there are more training instances for the model, and similarly when the dataset is large, we can choose a smaller value of $k$ (such as $k=5$) both to reduce the computational cost and to obtain lower variance in our estimate of the model's generalization ability, as the folds are more distinct from each other.

Once we find the optimal hyperparameter set, we retrain the model on the entire training set and obtain a final performance evaluation on the test set. For now, we will load and prepare the Breast Cancer Wisconsin dataset.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#import dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',
    header=None)

#create numpy arrays
X = df.loc[:, 2:].values
y = df.loc[:, 1]

#encode class labels, M is 1 and B is 0
le = LabelEncoder()
y = le.fit_transform(y)

#perform 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

We can improve on the above in the context of classification by using *stratified $k$-fold cross-validation*, in which the proportions of class labels are preserved across each fold. This can yield better bias and variance estimates, especially in the case of class imbalance. We can implement this via the `StratifiedKFold` iterator in sklearn:

In [2]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

#make logistic regression pipeline
pipe_lr = make_pipeline(StandardScaler(),
                       LogisticRegression(penalty='l2', max_iter=10000))

#instantiate StratifiedKFold class
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

#calculate scores the old-fashioned way
scores=[]
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    
#print mean and standard deviation of cv accuracy
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print("CV Accuracy: {mean} +/- {std}".format(mean=mean_acc, std=std_acc))

CV Accuracy: 0.9692753623188406 +/- 0.024367371956677097


We can also implement a $k$-fold cross-validation scorer in sklearn using `cross_val_score` which allows us to produce the scores above less verbosely. If the estimator is a classifier, the `y` parameter is either binary or multiclass, and the input to `cv` is an integer or is `None`, then `StratifiedKFold` is used. In all other cases, `KFold` is used.

In [4]:
from sklearn.model_selection import cross_val_score

#compute cv accuracy using cross_val_score
scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print("CV Accuracy: {mean} +/- {std}".format(mean=np.mean(scores), std=np.std(scores)))

CV Accuracy: 0.9692753623188406 +/- 0.024367371956677097


Observe that we got the exact same results using both methods. Also, note that we can control how many CPUs the cross-validation is distributed across by setting the `n_jobs` parameter equal to the number of CPUs in our machine (if there are multiple available). For example, by setting `n_jobs=-1`, we can use all available CPUs to do the computation in parallel.

# Hyperparameter Optimization Methods

While certain parameters are optimized during the training process (such as the weights of a neural network), others are chosen before training begins. These latter parameters are called *hyperparameters*, and need to be tuned in order to find the best model for a given dataset. We can use validation curves to come up with an initial range of values for the hyperparameters which we want to optimize. However, to find the optimal combination of hyperparameters, we need to perform a search over some set of hyperparameter combinations.

### Tuning Hyperparameters via Grid Search 

Grid search is a brute-force, exhaustive search over all possible combinations of hyperparameters in a set that we specify. This is extremely computationally expensive, as there are combinatorially many hyperparameter configurations to go through. We implement this in sklearn using an SVM model:

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

#make svc pipeline
pipe_svc = make_pipeline(StandardScaler(),
                        SVC(random_state=42))

#create hyperparameter grid
param_range = [0.0001, 0.001, 0.01, 0.1, 1., 10., 100., 1000.]
param_grid = [{'svc__C': param_range,
              'svc__kernel': ['linear']},
             {'svc__C': param_range,
             'svc__gamma': param_range,
             'svc__kernel': ['rbf']}]

#perform grid search over hyperparameter grid using 10-fold CV
gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 refit=True,
                 n_jobs=-1)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9757004830917875
{'svc__C': 100.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}


Now that we've found the optimal hyperparameters based on a $10$-fold CV, we can use these optimal hyperparameters to retrain the model on the entire training set, and then evaluate this model's performance on the test set.

In [12]:
#extract optimal model and evaluate on test set
opt_svc = gs.best_estimator_
#opt_svc.fit(X_train, y_train)
print("Test accuracy: {}".format(opt_svc.score(X_test, y_test).round(3)))

Test accuracy: 0.956


### Tuning Hyperparameters More Widely with Randomized Search

In randomized search, we draw hyperparameter combinations randomly from a distribution over the hyperparameter space or from a discrete set of hyperparameter options. While this is not an exhaustive search over the hyperparameter space, we get to explore a wider range of hyperparameter combinations in a more computationally effective manner.

While we could use the same discrete set of hyperparameters `param_range` that we defined for grid search above, the true power of the `RandomizedSearchCV` class in sklearn is that we can instead randomize over a probability distribution. So, we can use a loguniform distribution rather than the regular uniform distribution to ensure that, in the limit, the same number of samples will be drawn from the $[0.0001, 0.001]$ range as from the $[10, 100]$ range.

In [13]:
import scipy.stats

#create parameter range with loguniform distribution
param_range = scipy.stats.loguniform(0.0001, 1000.0)

#check behavior of the distribution
np.random.seed(42)
param_range.rvs(10)

array([4.18582273e-02, 4.51856095e+02, 1.33032451e+01, 1.55099140e+00,
       1.23631883e-03, 1.23583828e-03, 2.55026485e-04, 1.15673272e+02,
       1.61363417e+00, 9.04707196e+00])

Now we can apply the `RandomizedSearchCV` class to the SVC we created earlier.

In [17]:
from sklearn.model_selection import RandomizedSearchCV

#create parameter grid
param_grid = [{'svc__C': param_range,
              'svc__kernel': ['linear']},
             {'svc__C': param_range,
             'svc__gamma': param_range,
             'svc__kernel': ['rbf']}]

#perform randomized search over the param_grid distribution
rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        n_iter=100,
                        cv=10,
                        random_state=42,
                        n_jobs=-1)
rs = rs.fit(X_train, y_train)
print(rs.best_score_.round(3))
print(rs.best_params_)

0.976
{'svc__C': 2.3468121983173633, 'svc__gamma': 0.011733722171268566, 'svc__kernel': 'rbf'}


Finally, let's see if the randomized search CV results in a better performance on the test set (indeed, it does).

In [18]:
#extract optimal model and evaluate on test set
opt_svc = rs.best_estimator_
print('Test Accuracy: {}'.format(opt_svc.score(X_test, y_test).round(3)))

Test Accuracy: 0.982


### Resource-Efficient Hyperparameter Search with Successive Halving

Given a large set of potential hyperparameter configurations, successive halving successively discards unpromising configurations until only one remains. The procedure can be summarized as follows:
1. Draw a large set of candidate configurations via random sampling.
2. Train the models with a small subset of the training set.
3. Discard the bottom $50\%$ of configurations based on predictive performance.
4. Repeat steps 2 and 3 using a larger subset of the training set to train the models each time, until only one configuration is left.

In [20]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

#perform successive halving
hs = HalvingRandomSearchCV(pipe_svc,
                          param_distributions=param_grid,
                          n_candidates='exhaust',
                          resource='n_samples',
                          factor=1.5,
                          random_state=42,
                          n_jobs=-1)
hs = hs.fit(X_train, y_train)
print(hs.best_score_.round(3))
print(hs.best_params_)

0.968
{'svc__C': 0.06295450333384245, 'svc__kernel': 'linear'}


We set `factor=1.5` to keep $66\%$ of the remaining candidates each round (if we had set `factor=2`, we would keep $50\%$ of candidates). Now we again check the performance on the test set.

In [21]:
#extract best model and evaluate on test set
opt_hs = hs.best_estimator_
print("Test accuracy: {}".format(opt_hs.score(X_test, y_test).round(3)))

Test accuracy: 0.982


We observe that successive halving also performs very well, and similarly to randomized search CV.

### Nested Cross-Validation

The idea behind nested cross-validation is to perform model selection before we then optimize the hyperparameters of the best type of model. To do this, we partition the training set into $k_1$ folds, as we did before, but now we perform a $k_2$-fold CV within each mini-training set, and evaluate each one on the corresponding test fold. This gives us an (almost) unbiased estimate of a given model's ability to generalize to the test set. After we repeat this procedure for each potential model, we choose the model with the best estimated CV accuracy. We then perform a normal grid or randomized search CV to optimize the hyperparameters of this optimal model.

In sklearn, we can perform nested cross-validation with grid search on the SVC model as follows:

In [22]:
k1 = 10
k2 = 3

#create parameter grid
param_range = [0.0001, 0.001, 0.01, 0.1, 1., 10., 100., 1000.]
param_grid = [{'svc__C': param_range,
              'svc__kernel': ['linear']},
             {'svc__C': param_range,
             'svc__gamma': param_range,
             'svc__kernel': ['rbf']}]

#create grid search object which we will apply on each mini-training set
gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=k2)

#now perform outer loop, we iterate over all mini-training sets and apply gs object
scores = cross_val_score(gs, X_train, y_train, 
                        scoring='accuracy', cv=10)
print("CV accuracy for SVC: {mean} +/- {std}".format(mean=np.mean(scores), std=np.std(scores)))

CV accuracy for SVC: 0.9602415458937198 +/- 0.03097673292462225


The returned average cross-validation accuracy gives us a good estimate of what to expect if we tune the hyperparameters of a model and use it on unseen data. Now, we can perform the exact same procedure on a different classifier. We will use a decision tree for this example, and we will only tune its depth parameter:

In [23]:
from sklearn.tree import DecisionTreeClassifier

#create grid search object which we will apply on each mini-training set
gs = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                 param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, None]},
                 scoring='accuracy',
                 cv=k2)

#now perform outer loop, we iterate over all mini-training sets and apply gs object
scores = cross_val_score(gs, X_train, y_train,
                        scoring='accuracy', cv=10)
print("CV accuracy for SVC: {mean} +/- {std}".format(mean=np.mean(scores), std=np.std(scores)))

CV accuracy for SVC: 0.9183574879227054 +/- 0.044612683047718524


From the above, we see that we can expect the SVC to perform better on unseen data than the decision tree. Thus, we should choose the SVC to be the model whose hyperparameters we optimize for, which we can do using any of the methods already described above.